# How to compute with Tannakian symbols and multiplicative functions 


This notebook is meant to accompany the article _New perspectives on multiplicative functions_ by Espeseth, Holmstrom and Vik. 

We begin with some motivation, followed by an overview of what is currently implemented in Sage, divided into the following main sections:

* TS of a commutative monoid
* TS of a commutative ring
* TS of the complex numbers
* Methods for multiplicative functions

## Motivation

Let $Mult(\mathbb{C})$ be the set of all multiplicative functions (from the positive integers to the complex numbers). This set contains for example the Euler phi function, the Mobius mu function, the divisor functions $\sigma_k$, the Liouville function, and many other classical number-theoretic functions. In addition, every number-theoretic object with an L-function (such as a motive, a scheme, a Galois representation or an automorphic representation) also has an associated multiplicative function, obtained by reading off the coefficients of the Dirichlet series describing the L-function. Finally, many number-theoretic functions which are not multiplicative, can be interpreted within the framework of multiplicative functions, if we allow more general target rings than $\mathbb{C}$. 

References for the first paragraph: [Wikipedia: L-function](https://en.wikipedia.org/wiki/L-function), [Terence Tao: Derived multiplicative functions](https://terrytao.wordpress.com/2014/09/24/derived-multiplicative-functions/)

The set of multiplicative functions comes with many binary operations, such as Dirichlet convolution, unitary convolution, and the usual (pointwise) product of functions. It also carries many unary operations studied in the literature, such as the k'th convolute, the norm and higher norm operators of Redmond and Sivaramakrishnan, operators given by precomposition with $n \mapsto n^k$, and many others. A natural question is this: _What kind of algebraic structure is $Mult(\mathbb{C})$_?

In a paper (which is currently in fairly readable draft form) we review all operations studied in the literature, and show that they are all part of a rich algebraic structure which is closely related to the notion of a lambda-ring, first invented by Grothendieck. This unifies and clarifies many previously unrelated phenomena in number theory.

The notion of a lambda-ring appears naturally in many different parts of mathematics:
* The representation ring of a finite group is a lambda-ring
* More generally, the Grothendieck ring of a symmetric monoidal abelian category is a lambda-ring
* Any binomial ring (in the sense of Hall) is a lambda-ring
* The topological K-theory of a topological space is a lambda-ring
* Any commutative monoid algebra is a lambda-ring
* There are close relations between lambda-rings and the theory of Witt vectors

References: [Wikipedia: Lambda-ring](https://en.wikipedia.org/wiki/%CE%9B-ring), [Yau: Lambda-rings](https://books.google.co.uk/books/about/Lambda_rings.html?id=d7vKnjxyvxQC).

Lambda-rings are not currently implemented in Sage. We are not aiming for an abstract implementation of lambda-rings (although this could perhaps be interesting), but rather a practical implementation of _Tannakian symbols_, which is a tool for computations in lambda-rings, analogous to the way matrices are a tool for computations in associative algebras, and polynomials are a tool for computations in commutative rings.

## Very brief review of Tannakian symbols

For a more detailed discussion of what Tannakian symbols are good for and which lambda-rings can be described in this language, we refer to the article. Here we collect the main points very briefly.

* For any commutative monoid M, we can form TS(M), which is a commutative ring equipped with additional operations making in into a lambda-ring. 
* When $M$ is just a commutative monoid (with no additional structure), $TS(M)$ is just the same thing as the monoid algebra $\mathbb{Z}[M]$.
* The elements of $TS(M)$ are called Tannakian symbols. Such an element as an ordered pair of disjoint finite multisets with elements taken from $M$. We write an element on the form $\frac{A}{B}$ where $A$ and $B$ are multisets. For example: 
$$ \frac{ \{2, 5, 5 \} }{ \{1, -1 \} }  $$
is a Tannakian symbol (with entries taken from $\mathbb{Z}$).
* In the Tannakian symbol $\frac{A}{B}$, we refer to $A$ as the _upstairs_ multiset, and to $B$ as the _downstairs_ multiset.
* Any commutative ring $R$ is in particular a multiplicative monoid, and hence we can form $TS(R)$. This is not just a lambda-ring, but carries an even richer structure, with many different algebraic operations (in the article, we introduce axioms for "double Adams algebras", which describe these operations). 
* If the ring $R$ happens to be the complex numbers, even more operations arise naturally, many of which are described below.
* TS is a functor, in the simplest setting from the category of commutative monoids to the category of lambda-rings.
* There is a correspondence between a certain class of multiplicative functions (the rational ones) and a certain class of Tannakian symbols, and from this it follows that there is a close relation between operations on Tannakian symbols and operations on multiplicative functions.

## Getting started with computations

In any Jupyter notebook or Sage worksheet, start by importing the following files: 

In [ ]:
%runfile ../zetatypes/util/lazylist.py
%runfile ../zetatypes/ts/monoid.py
%runfile ../zetatypes/ts/ring.py
%runfile ../zetatypes/util/berlekamp.py
%runfile ../zetatypes/ts/complex.py

In a Sage worksheet, use "%attach" instead of "%runfile".

## TS of a commutative monoid

Now to some examples. 

We create four variables, which can be multiplied in Sage.

In [ ]:
a = var('a'); b = var('b'); c = var('c'); d = var('d');

Let $F$ be a monoid in which these variables live. We can now create the lambda-ring $TS(F)$.

In [ ]:

#This code is commented out - can use it later if we find a way of improving the printing of monoid elements.
#reset('a'); reset('b'); reset('c'); reset('d');
#F.<a,b,c,d> = Monoids().Commutative().free(); 
#print a*b
#To get TS of the free commutative monoid on a, b, c, d, we would do this:
#TSF = TannakianSymbols(ZZ, F)  
#
#The above command works perfectly well, but because of the ugly printing issue, we use a work-around instead, 
#with the ring Z[a, b, c, d] as our monoid.
TSF = TannakianSymbols(ZZ, ZZ)

We can multiply indivindual elements of the monoid:

In [ ]:
a * b * c * a * c^4

To build a Tannakian symbol from a string, we use TSF.parseSymbol, and it will be convenient to introduce an abbreviation for this command.

In [ ]:
ts = TSF.parseSymbol

Now let's create a few Tannakian symbols in the ring $TS(F)$. We call these symbols $X, Y$ and $Z$.


In [ ]:
X = ts("{a, b}/{c, d }")

For the empty multiset, we may use the Norwegian letter Ø or simply {}.

In [ ]:
Y = ts("{a}/Ø")

Since we use multisets and not just sets, we may have repeated elements:

In [ ]:
Z = ts("{a}/{c, c, c}")

We can print Tannakian symbols.

In [ ]:
print X; print Y; print Z

We can add and multiply Tannakian symbols. These operations are (for good mathematical reasons) referred to as the "direct sum" and the "tensor product" in our article and elsewhere.

In [ ]:
X + X

In [ ]:
X + Y

In [ ]:
X * Y

In [ ]:
Z^2

We can subtract a symbol from another symbol. The zero element of the ring $TS(F)$ is $\emptyset / \emptyset$.

In [ ]:
X - X

We can extract a list of the elements upstairs, or a list of the elements downstairs:

In [ ]:
Z.upstairs()

In [ ]:
Z.downstairs()

We can also compute a few invariants of a Tannakian symbol, namely _even dimension_ (number of elements upstairs), _odd dimension_ (the number of elements downstairs, and the _augmentation_ (the difference between the even and the odd dimension).

In [ ]:
print "The symbol Z is:", Z
print "Even dimension:", Z.evendimension() 
print "Odd dimension:", Z.odddimension() 
print "Augmentation:", Z.augmentation() 

The ordered pair of integers defined by the even and the odd dimension can be produced directly using the "superdimension" command":

In [ ]:
print "Superdimension:", Z.superdimension()

We have also introduced the determinant of a symbol, which is defined whenever the elements downstairs have inverses. For example, if the monoid happens to be a group, the determinant is defined for all Tannakian symbols.

In [ ]:
print "Determinant:", Z.determinant()

We are not explaining the axioms for lambda-rings here, but any lambda-ring carries four infinite sequences of unary operations. We illustrate each of these with a few examples.

First: lambda operations. In standard mathematical notation, for every non-negative integer $k$ there is a unary operation $\lambda^k$ from $TS(M)$ to $TS(M)$. Below we print the symbol $X$, and then we compute $\lambda^k(X)$ for a few small values of $k$.

In [ ]:
#Redefining X
X = ts("{a,b,c}/Ø")
print "The symbol X is:", X
print "Applying lambda-operations for $k = 1, 2, ..., we get:"
print X.lambdaoperation(1)
print X.lambdaoperation(2)
print X.lambdaoperation(3)
print X.lambdaoperation(4)

Secondly: Adams operations. In standard mathematical notation, for every non-negative integer $k$, there is a unary operation $\psi^k$ from $TS(M)$ to $TS(M)$. These operations are different from the others in that they are ring endomorphisms rather than just functions. Below we print the symbol $X$, and then we compute $\psi^k(X)$ for a few small values of $k$.

In [ ]:
print X

print X.adamsoperation(1)
print X.adamsoperation(2)
print X.adamsoperation(3)
print X.adamsoperation(4)

Thirdly: gamma operations. In standard mathematical notation, for every non-negative integer $k$, there is a unary operation $\gamma^k$ from $TS(M)$ to $TS(M)$. Below we print the symbol $X$, and then we compute $\gamma^k(X)$ for a few small values of $k$.

In [ ]:
print X

print X.gammaoperation(1)
print X.gammaoperation(2)
print X.gammaoperation(3)
print X.gammaoperation(4)

Finally, we also have symmetric power operations. For every non-negative integer $k$, there is a unary operation $Sym^k$ from $TS(M)$ to $TS(M)$. Again, we print the symbol $X$, and then we compute $Sym^k(X)$ for a few small values of $k$.

In [ ]:
print X

print X.symmetricpoweroperation(1)
print X.symmetricpoweroperation(2)
print X.symmetricpoweroperation(3)
print X.symmetricpoweroperation(4)

All the commands we have used so far are defined in the file MonoidTS.sage. This file contains the definition of the class TannakianSymbols, which extends the class CombinatorialFreeModule. Feel free to inspect the source code and ask Torstein Vik if anything is unclear :-)

## TS of a commutative ring

As we have already mentioned, we may consider $TS(R)$, where $R$ is not just a commutative (multiplicative) monoid, but a commutative ring. In this richer setting, we have a number of additional algebraic operations and invariants. At the moment, only three commands are implemented in the generality of general commutative rings, and everything else is done for complex numbers only (see next section).

To get access to these new commands, we must define TSR using the class "RingTannakianSymbols", and redefine $X, Y, Z$ as elements of TSR. We also define the convenient abbreviation "ts" again.

In [ ]:
TSR = RingTannakianSymbols(ZZ[a, b, c, d])

ts = TSR.parseSymbol

X = ts("{a, b}/{c, d }")
Y = ts("{a}/Ø")
Z = ts("{a}/{c, c, c}")

The simplest new invariant is given by the _trace_ of a Tannakian symbol. It is the sum of the elements upstairs, minus the sum of the elements downstairs.

In [ ]:
print "Trace of Z:", Z.trace()
print "Trace of X:", X.trace()

The trace is an element in a certain infinite sequence associated to a Tannakian symbol, called _the sequence of Bell coefficients_. This is a linearly recursive sequence of elements in the ring $R$, whose zero'th element is 1, whose first element is the trace, and in general has the property that the power series corresponding to the sequence is the same as the rational power series defined by a certain rational expression which contains the elements from the Tannakian symbol. See the article for more details.

Examples:

In [ ]:
print "First 3 Bell coefficients of Z:", Z.getBellCoefficients(3)

Note that an object of the form Z.getBellCoefficients behaves like an infinite list. This feature relies on the class LazyList which was imported at the very top of this document. Printing this object currently produces a printout of the first 20 elements (this is an arbitrary convention that seemed reasonable).

We use the symbol Y to illustrate this point:

In [ ]:
print "The symbol Y is", Y
bc = Y.getBellCoefficients()
print bc[3]
print bc[30]
print bc

Replacing Y by a more complicated symbol is fine, but gives (unsurprisingly) a more complicated output.

In [ ]:
U = ts("{a}/{b}")
print "The symbol U is", U
bc = U.getBellCoefficients()
print bc[3]
print bc[30]
print bc

There is also another similar sequence associated to a Tannakian symbol, referred to as the sequence of point counts. This sequence can be computed by the command "getPointCounts".

In [ ]:
print "The symbol U is", U
bc = U.getPointCounts()
print bc[3]
print bc[30]
print bc

We can of course combine these methods with some of those defined earlier. Here are a few examples:

In [ ]:
(U+Y).symmetricpoweroperation(5)

In [ ]:
(Z-X).gammaoperation(3)

We can investigate how the superdimension of $\gamma^k \big( \frac{\{a, b\}}{\emptyset} \big)$ varies with $k$:

In [ ]:
W = ts("{a, b}/{}")
print W
for k in range(1,10):
    print (W.gammaoperation(k)).superdimension()

There is a close relationship between these various operations and the theory of symmetric polynomials, as illustrated in the following examples:

In [ ]:
W = ts("{a, b, c}/{}")
print "We use the symbol W as input. W equals", W
print "Applying lambda-operations for $k = 1, 2, 3, 4$, in each case followed by the trace function, we get:"
for k in range(1, 5):
    print "k =", k, ":",  (W.lambdaoperation(k)).trace()

In [ ]:
W = ts("{a, b, c}/{}")
print "We use the symbol W as input. W equals", W
print "Applying Adams operations for $k = 1, 2, 3, 4$, in each case followed by the trace function, we get:"
for k in range(1, 5):
    print "k =", k, ":",  (W.adamsoperation(k)).trace()

In [ ]:
W = ts("{a, b, c}/{}")
print "We use the symbol W as input. W equals", W
print "Applying symmetric power operations for $k = 1, 2, 3, 4$, in each case followed by the trace function, we get:"
for k in range(1, 5):
    print "k =", k, ":",  (W.symmetricpoweroperation(k)).trace()

In [ ]:
W = ts("{a, b, c}/{}")
print "We use the symbol W as input. W equals", W
print "Applying gamma-operations for $k = 1, 2, 3, 4$, in each case followed by the trace function, we get:"
for k in range(1, 5):
    print "k =", k, ":",  (W.gammaoperation(k)).trace()

The methods that have been described here for $TS(R)$ are all defined in the file RingTS.sage.

## TS of the complex numbers

We begin by constructing $TS(\mathbb{C}^{\times})$, which is the functor $TS$ applied to the commutative monoid of non-zero complex numbers. We also redefine the command "ts" and introduce a few example symbols.

In [ ]:
TSC = ComplexTannakianSymbols()

ts = TSC.parseSymbol

X = ts("{2, 3}/{-1, -1 }")
Y = ts("{1+I, 1-I}/Ø")
Z = ts("{-2}/{I, I, 4*I}")

All the commands introduced above also work for complex Tannakian symbols. A few examples:

In [ ]:
print X * Y

In [ ]:
print Z
print Z.determinant()
print Z.trace()

In [ ]:
print Z.symmetricpoweroperation(3)

In [ ]:
print Z.getBellCoefficients()

We also have a plethora of new operations. Some of these really exist in greater generality (for TS(R) when $R$ is a general commutative ring, or a commutative ring satisfying some extra properties), but have only been implemented for complex numbers. 

The main difficulty when implementing these operations is that some of them rely on the Berlekamp-Massey algorithm, which takes a linearly recursive sequence as input, and returns two polynomials whose quotient is a generating series for the input sequence. The version of Berlekamp-Massey that we use was written by Magnus Hellebust Haaland and Olav Hellebust Haaland, and works well when the input is an integer sequence of fairly low recursion degree (we should specify here what this means, it seems anything under 20 is fine, must do some more testing here).

#### Box sum, box product, and showplot

To begin with, we have the "box sum" and the "box product". Both of these operations rely on Berlekamp-Massey.

In [ ]:
TSC.boxsum(X, X)

In [ ]:
TSC.boxproduct(X, Y)

In [ ]:
W = TSC.boxsum(X+X, Y)
print W

If you feel that this looks a bit messy, you may print the elements one at a time instead:

In [ ]:
print "Upstairs:"
for x in W.upstairs(): print x
print "Downstairs:"
for x in W.downstairs(): print x

If you prefer a plot of the elements in the complex plane, use the showplot command. Here the upstairs elements are shown in blue (by default), while the downstairs element are shown in red. A larger circle indicates that the multiplicity is higher than 1.

In [ ]:
W.showplot()